In [ ]:
# In this challenge, you need to create a machine learning model that will classify SMS messages as either "ham" or "spam".
# A "ham" message is a normal message sent by a friend. A "spam" message is an advertisement or a message sent by a company.

# You should create a function called predict_message that takes a message string as an argument and returns a list.
# The first element in the list should be a number between zero and one that indicates the likeliness of "ham" (0) or "spam" (1).
# The second element in the list should be the word "ham" or "spam", depending on which is most likely.

# For this challenge, you will use the SMS Spam Collection dataset. The dataset has already been grouped into train data and test data.

# The first two cells import the libraries and data. The final cell tests your model and function. Add your code in between these cells.

In [2]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
import re
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.text import Tokenizer
# You may need to install libraries
# ! pip install pandas
# ! pip install nltk
# ! pip install scikit-learn

# Import libraries
import string
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
print(tf.__version__)

2.15.0-dev20231009


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

train_text = pd.read_csv(train_file_path, sep='\t', header=None)
train_text.columns = ['label', 'message']
test_text = pd.read_csv(test_file_path, sep='\t' ,  header=None)
test_text.columns = ['label', 'message']



#Check for balance in train set
spam_messages = train_text[train_text['label'] == "spam"]
ham_messages = train_text[train_text['label'] == "ham"]
print(f"Number of spam messages: {len(spam_messages)}")
print(f"Number of ham messages: {len(ham_messages)}")

#Get labels and texts separetely
train_data = train_text['message']
train_labels = train_text['label']

test_data = test_text['message']
test_labels = test_text['label']


data = train_text.append(test_text, ignore_index=True)
print(data.shape)
print(data.head())
tags = data["label"]
texts = data["message"]
num_max = 1000
le = LabelEncoder()
tags = le.fit_transform(tags)
tok = Tokenizer(num_words=num_max)
tok.fit_on_texts(texts)

mat_data = tok.texts_to_matrix(texts,mode='count')
mat_train = tok.texts_to_matrix(train_text,mode='count')
mat_test = tok.texts_to_matrix(test_text,mode='count')

max_len = 150
x_train = tok.texts_to_sequences(train_text["message"])
x_test = tok.texts_to_sequences(test_text["message"])
encoded_train_processed = keras.preprocessing.sequence.pad_sequences(x_train,maxlen=max_len)
encoded_test_processed = keras.preprocessing.sequence.pad_sequences(x_test,maxlen=max_len)

train_labels_ = train_text["label"]
train_labels = le.fit_transform(train_labels_)
test_labels_ = test_text["label"]
test_labels = le.fit_transform(test_labels_)

print(train_labels.shape)
print(encoded_train_processed.shape)


--2023-10-09 21:32:30--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 104.26.2.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv.42’

train-data.tsv.42   100%[===================>] 349.84K  --.-KB/s    in 0.06s   

2023-10-09 21:32:30 (5.79 MB/s) - ‘train-data.tsv.42’ saved [358233/358233]

--2023-10-09 21:32:30--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 104.26.2.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-separated-values]
Saving to: ‘valid-data.tsv.42’

valid-data.tsv.42   100%[=====

<ipython-input-4-a1e376cd6fe3>:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = train_text.append(test_text, ignore_index=True)


(4179,)
(4179, 150)


In [5]:
MAX_LEN = 150
BATCH_SIZE = 32
VOCAB_SIZE = 5000


#Creating the Model
model = tf.keras.Sequential([
      tf.keras.layers.Embedding(VOCAB_SIZE, 32 ),
      tf.keras.layers.LSTM(32),
      tf.keras.layers.Dense(1, activation = 'sigmoid')
  ])

model.summary


<bound method Model.summary of <keras.src.engine.sequential.Sequential object at 0x7a36adeba380>>

In [6]:
model.compile(loss = 'binary_crossentropy', optimizer = 'rmsprop', metrics = ['acc'])

history = model.fit(encoded_train_processed, train_labels, epochs = 10, validation_split = 0.2)

Epoch 1/10
105/105 [==============================] - 13s 103ms/step - loss: 0.2753 - acc: 0.9031 - val_loss: 0.1274 - val_acc: 0.9593
Epoch 2/10
105/105 [==============================] - 8s 77ms/step - loss: 0.0826 - acc: 0.9788 - val_loss: 0.0583 - val_acc: 0.9856
Epoch 3/10
105/105 [==============================] - 10s 93ms/step - loss: 0.0515 - acc: 0.9868 - val_loss: 0.0522 - val_acc: 0.9904
Epoch 4/10
105/105 [==============================] - 7s 71ms/step - loss: 0.0399 - acc: 0.9886 - val_loss: 0.0505 - val_acc: 0.9868
Epoch 5/10
105/105 [==============================] - 11s 103ms/step - loss: 0.0319 - acc: 0.9907 - val_loss: 0.0483 - val_acc: 0.9880
Epoch 6/10
105/105 [==============================] - 8s 81ms/step - loss: 0.0266 - acc: 0.9922 - val_loss: 0.0558 - val_acc: 0.9844
Epoch 7/10
105/105 [==============================] - 11s 100ms/step - loss: 0.0229 - acc: 0.9940 - val_loss: 0.0542 - val_acc: 0.9856
Epoch 8/10
105/105 [==============================] - 11s 102m

In [7]:
results = model.evaluate(encoded_test_processed, test_labels)
print(results)

44/44 [==============================] - 1s 16ms/step - loss: 0.0586 - acc: 0.9849
[0.058634236454963684, 0.9849137663841248]


In [9]:
#to text new data, new setences, its necessary to encode again. Lets bring back our econding process
new_data = "how are you doing today?"
new_data = [new_data]
print(new_data)

inputstr = tok.texts_to_sequences(new_data)
pred = model.predict(inputstr)

# print(result_)
p = (pred[0][0])
print([p,("ham" if p<0.5 else "spam")])

['how are you doing today?']
1/1 [==============================] - 1s 513ms/step
[0.0019632364, 'ham']


In [11]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  new_data = pred_text
  new_data = [new_data]
  print(new_data)

  inputstr = tok.texts_to_sequences(new_data)
  pred = model.predict(inputstr)

  # print(result_)
  p = (pred[0][0])
  class_ = ("ham" if p<0.5 else "spam")
  # print([p,("ham" if p<0.5 else "spam")])

  return  [p, class_]

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

['how are you doing today?']
1/1 [==============================] - 0s 53ms/step
[0.0019632364, 'ham']


In [12]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    print(msg)
    prediction = predict_message(msg)
    print(prediction)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


how are you doing today
['how are you doing today']
1/1 [==============================] - 0s 55ms/step
[0.0019632364, 'ham']
sale today! to stop texts call 98912460324
['sale today! to stop texts call 98912460324']
1/1 [==============================] - 0s 37ms/step
[0.81110317, 'spam']
i dont want to go. can we try it a different day? available sat
['i dont want to go. can we try it a different day? available sat']
1/1 [==============================] - 0s 476ms/step
[0.00047631626, 'ham']
our new mobile video service is live. just install on your phone to start watching.
['our new mobile video service is live. just install on your phone to start watching.']
1/1 [==============================] - 0s 54ms/step
[0.99658614, 'spam']
you have won £1000 cash! call to claim your prize.
['you have won £1000 cash! call to claim your prize.']
1/1 [==============================] - 0s 41ms/step
[0.9988115, 'spam']
i'll bring it tomorrow. don't forget the milk.
["i'll bring it tomorrow. don't f